In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.0 MB/s 
     |████████████████████████████████| 5.8 MB 23.3 MB/s 
     |████████████████████████████████| 212 kB 60.0 MB/s 
     |████████████████████████████████| 182 kB 60.8 MB/s 
     |████████████████████████████████| 132 kB 53.2 MB/s 
     |████████████████████████████████| 127 kB 50.2 MB/s 
     |████████████████████████████████| 7.6 MB 41.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786 Project/Remy

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ECE1786 Project/Remy


In [3]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

In [4]:
model_checkpoint = "gpt2"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
recipes = []
df = pd.read_pickle("/content/drive/MyDrive/ECE1786 Project/Remy/cocktail_dataset.pkl")  
for i in range(len(df.index)):
    recipe = "DRINK RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nDRINK RECIPE INGREDIENTS\n"
    skip = False
    for ingredient in df.loc[i, "Ingredients"]:
        # remove asterisks
        ingredient.replace('*', '')
        if "750" in ingredient:
            #skip batch drink recipes
            skip = True
        recipe += ingredient + "\n"
    if skip:
        continue
    recipe += "\nDRINK RECIPE INSTRUCTIONS\n" 
    for instruction in  df.loc[i, "Instructions"]:
        if instruction.startswith("*"):
            continue
        recipe += instruction + "\n"
    recipes.append(recipe)
num_recipes = len(recipes)
df = pd.read_pickle("/content/drive/MyDrive/ECE1786 Project/Remy/cake_sorted.pkl")  
for i in range(len(df.index)):
    recipe = "CAKE RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nCAKE RECIPE INGREDIENTS\n"
    skip = False
    for ingredient in df.loc[i, "Ingredients"]:
        '''
        # remove asterisks
        ingredient.replace('*', '')
        if "750" in ingredient:
            #skip batch drink recipes
            skip = True
        '''
        if "®" in ingredient:
            #skip name brand ingredients
            skip = True
        recipe += ingredient + "\n"
    if skip:
        continue
    recipe += "\nCAKE RECIPE INSTRUCTIONS\n" 
    for instruction in  df.loc[i, "Instructions"]:
        if instruction.startswith("*"):
            continue
        recipe += instruction + "\n"
    recipes.append(recipe)
    if len(recipes) >= 2*num_recipes:
        # make sure number of samples is the same
        break
tokenizer.pad_token = tokenizer.eos_token
data = tokenizer(recipes, padding='longest', truncation=True)
data["labels"] = data["input_ids"].copy()

dataset = Dataset.from_dict(data)
print(len(dataset))
print(dataset[5])
print(dataset)



584
{'input_ids': [7707, 17248, 19644, 4061, 36, 36751, 198, 7738, 18531, 220, 198, 198, 7707, 17248, 19644, 4061, 36, 3268, 10761, 1961, 40, 15365, 198, 17, 24405, 47553, 29219, 198, 16, 14, 17, 25799, 1667, 292, 354, 2879, 300, 2350, 333, 198, 16, 14, 17, 25799, 350, 2797, 304, 14937, 198, 38, 1501, 680, 25, 1667, 292, 354, 2879, 23612, 198, 198, 7707, 17248, 19644, 4061, 36, 3268, 46126, 11053, 198, 4550, 262, 47553, 29219, 11, 1667, 292, 354, 2879, 300, 2350, 333, 290, 350, 2797, 304, 14937, 656, 257, 17090, 5405, 351, 4771, 290, 11240, 1566, 880, 12, 354, 2967, 13, 198, 1273, 3201, 656, 257, 24554, 5405, 13, 198, 38, 1501, 680, 351, 257, 1667, 292, 354, 2879, 23612, 13, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 502

In [12]:
print(tokenizer.decode(dataset[352]["input_ids"]))


CAKE RECIPE NAME
Coconut Sour Cream Pound Cake 

CAKE RECIPE INGREDIENTS
¼ teaspoon baking soda
1 tablespoon coconut extract
6  eggs
3 cups sifted cake flour
1 cup sour cream
1 cup vegetable shortening
3 cups white sugar

CAKE RECIPE INSTRUCTIONS
Preheat oven to 300 degrees F. Grease and flour a 9-inch tube pan.
Beat vegetable shortening and sugar together in a bowl using an electric mixer until mixture is creamy. Beat eggs into shortening mixture, one at a time, letting each egg blend in before adding the next. Beat in sour cream until smooth.
Gradually stir cake flour and baking soda into the moist ingredients until batter is smooth; stir in coconut extract. Scoop batter into prepared tube pan.
Bake in the preheated oven until a toothpick inserted into the middle of the cake comes out clean or with moist crumbs, 1 hour and 20 minutes.
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoft

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-both",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=50,
    per_device_train_batch_size=2
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_vers

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 584
  Num Epochs = 50
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 14600
  Number of trainable parameters = 124439808


Step,Training Loss
500,0.462800
1000,0.347200
1500,0.317800
2000,0.302100
2500,0.283200
3000,0.261800
3500,0.255100
4000,0.245600
4500,0.231700
5000,0.222000


Saving model checkpoint to gpt2-finetuned-both/checkpoint-292
Configuration saved in gpt2-finetuned-both/checkpoint-292/config.json
Model weights saved in gpt2-finetuned-both/checkpoint-292/pytorch_model.bin
Saving model checkpoint to gpt2-finetuned-both/checkpoint-584
Configuration saved in gpt2-finetuned-both/checkpoint-584/config.json
Model weights saved in gpt2-finetuned-both/checkpoint-584/pytorch_model.bin
Deleting older checkpoint [gpt2-finetuned-both/checkpoint-292] due to args.save_total_limit
Saving model checkpoint to gpt2-finetuned-both/checkpoint-876
Configuration saved in gpt2-finetuned-both/checkpoint-876/config.json
Model weights saved in gpt2-finetuned-both/checkpoint-876/pytorch_model.bin
Deleting older checkpoint [gpt2-finetuned-both/checkpoint-584] due to args.save_total_limit
Saving model checkpoint to gpt2-finetuned-both/checkpoint-1168
Configuration saved in gpt2-finetuned-both/checkpoint-1168/config.json
Model weights saved in gpt2-finetuned-both/checkpoint-1168

RuntimeError: ignored

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
#pick up where training stopped
model_checkpoint = "./gpt2-finetuned-both/checkpoint-11680"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
training_args = TrainingArguments(
    f"gpt2-finetuned-both",
    save_strategy="epoch",
    save_total_limit=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=10,
    per_device_train_batch_size=2
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 584
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 2920
  Number of trainable parameters = 124439808


Step,Training Loss
500,0.153700
1000,0.144100
1500,0.135300
2000,0.131200
2500,0.125400


Saving model checkpoint to gpt2-finetuned-both/checkpoint-292
Configuration saved in gpt2-finetuned-both/checkpoint-292/config.json
Model weights saved in gpt2-finetuned-both/checkpoint-292/pytorch_model.bin
Saving model checkpoint to gpt2-finetuned-both/checkpoint-584
Configuration saved in gpt2-finetuned-both/checkpoint-584/config.json
Model weights saved in gpt2-finetuned-both/checkpoint-584/pytorch_model.bin
Saving model checkpoint to gpt2-finetuned-both/checkpoint-876
Configuration saved in gpt2-finetuned-both/checkpoint-876/config.json
Model weights saved in gpt2-finetuned-both/checkpoint-876/pytorch_model.bin
Saving model checkpoint to gpt2-finetuned-both/checkpoint-1168
Configuration saved in gpt2-finetuned-both/checkpoint-1168/config.json
Model weights saved in gpt2-finetuned-both/checkpoint-1168/pytorch_model.bin
Saving model checkpoint to gpt2-finetuned-both/checkpoint-1460
Configuration saved in gpt2-finetuned-both/checkpoint-1460/config.json
Model weights saved in gpt2-fin

TrainOutput(global_step=2920, training_loss=0.13523212328349074, metrics={'train_runtime': 2153.2167, 'train_samples_per_second': 2.712, 'train_steps_per_second': 1.356, 'total_flos': 3051890933760000.0, 'train_loss': 0.13523212328349074, 'epoch': 10.0})

In [ ]:
prompt = "CAKE RECIPE NAME\n"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda:0")

outputs = model.generate(input_ids, do_sample=True, max_length=600,temperature = 0.5, top_p = 1)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CAKE RECIPE NAME
Caramel Pound Cake 

CAKE RECIPE INGREDIENTS
1 cup all-purpose flour
½ teaspoon baking soda
½ cup butter, softened
1  egg
1 cup milk
½ teaspoon salt
1 cup sour cream
1 cup unsweetened cocoa powder
1 teaspoon vanilla extract
1 cup white sugar

CAKE RECIPE INSTRUCTIONS
Preheat oven to 350 degrees F. Grease and flour a 9 inch tube pan.
In a large bowl, cream butter and sugar until light and fluffy. Add milk, one at a time, beating well after each addition. Add vanilla and baking soda. Beat 1 cup at a time into the butter mixture until blended.
Pour batter into the prepared tube pan. Bake in the preheated oven for 1 hour or until a toothpick inserted into the center of the cake comes out clean. Let cake cool in the pan for 10 minutes, then turn out onto a wire rack to cool completely.
To make Frosting: In a medium bowl, combine sugar, cocoa, and milk. Stir in flour and salt.



In [ ]:
from transformers import AutoModelForCausalLM
model_checkpoint = "./distilgpt2-finetuned-both/checkpoint-8000"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", use_fast=True)

OSError: ignored

In [ ]:
prompt = "RECIPE NAME\nBlueberry Cheesecake"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cpu")

outputs = model.generate(input_ids, do_sample=True, max_length=600,temperature = 0.9, top_p = 1)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RECIPE NAME
Blueberry Cheesecake 

RECIPE INGREDIENTS
2 ¼ cups all-purpose flour
2 teaspoons baking powder
3 large eggs
½ teaspoon ground cinnamon
½ teaspoon ground cloves
¾ cup packed brown sugar
¾ cup sour cream
½ cup unsalted butter, at room temperature 
1 teaspoon vanilla extract
1 1/2 cups white sugar

RECIPE INSTRUCTIONS
Preheat the oven to 350 degrees F. Grease and flour two 10 inch tube pans.
In a medium bowl, cream together the butter and sugar until light and fluffy. Beat in the eggs one at a time, then stir in the vanilla. Beat in the flour and cinnamon. Add the sour cream and mix well. Mix in the mashed bananas. Pour batter into prepared pans.
Bake in the preheated oven for 70 to 80 minutes, or until a toothpick inserted into the center of the cake comes out clean. Let cool in pans for 10 minutes, then turn out onto a wire rack and cool completely.

